In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder

In [3]:
train_df = pd.read_csv('/kaggle/input/titanic/train.csv')
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')

FileNotFoundError: [Errno 2] File /kaggle/input/titanic/train.csv does not exist: '/kaggle/input/titanic/train.csv'

In [ ]:
train_df1 = train_df.copy()
test_df1 = test_df.copy()

# Drop Ticket and Cabin columns


In [ ]:
train_df1 = train_df1.drop(['Ticket', 'Cabin'], axis=1)
test_df1 = test_df1.drop(['Ticket', 'Cabin'], axis=1)

# **Create Title coumn from Name column**

In [ ]:
def extract(name):
    title = re.search('([A-Za-z]+)\.', name)
    return title[1]

train_df1['Title'] = train_df1['Name'].apply(extract)
test_df1['Title'] = test_df1['Name'].apply(extract) 

In [ ]:
dict = {'Mr': 0, 'Miss': 1, 'Mrs': 2, 'Master': 3, 'Dr': 0, 'Rev': 0, 'Mlle': 2, 'Col': 0, 'Major': 0,
             'Mme': 2, 'Sir': 0, 'Jonkheer': 0, 'Capt': 0, 'Ms': 2, 'Countess': 2, 'Lady': 1, 'Don': 0}

train_df1['Title'] = train_df1['Title'].map(dict)
test_df1['Title'] = test_df1['Title'].map(dict)

test_df1.head()

In [ ]:
train_df1 = train_df1.drop(['Name', 'PassengerId'], axis=1)
test_df1 = test_df1.drop(['Name'], axis=1)
test_df1['Title'].fillna(2, inplace=True)

# Sex - convert to numeric value

In [ ]:
test_df1['Sex'] = test_df1['Sex'].apply(lambda x: 0 if x=='male' else 1)
train_df1['Sex'] = train_df1['Sex'].apply(lambda x: 0 if x=='male' else 1)

# Age - Fill missing values- median values by grouping('Sex', 'Pclass', 'Title')

In [ ]:
train_df1['Age'].fillna(train_df1.groupby(['Sex', 'Pclass'])['Age'].transform('median'), inplace=True)
test_df1['Age'].fillna(test_df1.groupby(['Sex', 'Pclass'])['Age'].transform('median'), inplace=True)

# Replace age with age bands

In [ ]:
# combine = [train_df1, test_df1]

# train_df1['AgeBand'] = pd.cut(train_df1['Age'], 5)
# test_df1['AgeBand'] = pd.cut(test_df1['Age'], 5)

# train_df1['Age2'] = train_df1['Age']
# test_df1['Age2'] = test_df1['Age']

# train_df1.head()

In [ ]:
# for dataset in combine:    
#     dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
#     dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
#     dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
#     dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
#     dataset.loc[ dataset['Age'] > 64, 'Age']

# test_df1.head()

In [ ]:
# train_df1 = train_df1.drop(['AgeBand'], axis=1)
# test_df1 = test_df1.drop(['AgeBand'], axis=1)

 # Embarked - fill and convert to numeric value

In [ ]:
# Find frequent value in Embarked
freq_port1 = train_df1.Embarked.value_counts(ascending=False).index[0]
freq_port2 = test_df1.Embarked.value_counts(ascending=False).index[0]

In [ ]:
train_df1['Embarked'] = train_df1['Embarked'].fillna(freq_port1)
test_df1['Embarked'] = test_df1['Embarked'].fillna(freq_port2)

In [ ]:
train_df1['Embarked'] = train_df1['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
test_df1['Embarked'] = test_df1['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

# Family_size - By compining Parch ans SibSp columns

In [ ]:
train_df1['Family_size'] = train_df1['Parch'] + train_df1['SibSp'] + 1
test_df1['Family_size'] = test_df1['Parch'] + train_df1['SibSp'] + 1

In [ ]:
train_df1['Family_size'] = train_df1['Family_size'].apply(lambda x: 0 if x >=3 else 1)
train_df1.drop(['Parch', 'SibSp'], axis=1, inplace=True)

test_df1['Family_size'] = test_df1['Family_size'].apply(lambda x: 0 if x >=3 else 1)
test_df1.drop(['Parch', 'SibSp'], axis=1, inplace=True)

# Fare - Test data- fill mean value¶

In [ ]:
train_df1['FareBand'] = pd.qcut(train_df['Fare'], 10)
train_df1['FareBand'] = train_df1['FareBand'].astype('category')
train_df1['Fare_cat'] = train_df1['FareBand'].cat.codes

In [ ]:
test_df1['FareBand'] = pd.qcut(test_df['Fare'], 10)
test_df1['FareBand'] = test_df1['FareBand'].astype('category')
test_df1['Fare_cat'] = test_df1['FareBand'].cat.codes

In [ ]:
train_df1 = train_df1.drop(['Fare', 'FareBand'], axis=1)
test_df1 = test_df1.drop(['Fare', 'FareBand'], axis=1)

In [ ]:
# test_df1['Title'] = test_df1['Title'].apply(np.int64) # convert Title in test_df1 into integer
# test_df1['Title'] = test_df1['Age'].apply(np.int64)

In [ ]:
train_df1.astype(int).head()

In [ ]:
test_df1.astype(int).head()

# Model, predict and solve

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier,AdaBoostClassifier,VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
X = train_df1.drop("Survived", axis=1)
y = train_df1["Survived"]
X_test  = test_df1.drop("PassengerId", axis=1).copy()


In [ ]:
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [ ]:
X.shape , X_test.shape

# KNn

In [ ]:
knn = KNeighborsClassifier(n_neighbors=30)
knn.fit(X_train, y_train)
Y_pred_knn = knn.predict(X_test)
acc_log = round(knn.score(X_valid, y_valid) * 100, 2)
acc_log

# Decision Tree

In [ ]:
dst = DecisionTreeClassifier()
dst.fit(X_train, y_train)
Y_pred_dst = dst.predict(X_test)
acc_log = round(dst.score(X_valid, y_valid) * 100, 2)
acc_log

# Adaboost

In [ ]:
ada = AdaBoostClassifier(dst, random_state=7)

# ada_param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
#               "base_estimator__splitter" :   ["best", "random"],
#               "algorithm" : ["SAMME","SAMME.R"],
#               "n_estimators" :[1,2],
#               "learning_rate":  [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3,1.5]}

ada_param_best = {"base_estimator__criterion" : ["entropy"],
              "base_estimator__splitter" :   ["random"],
              "algorithm" : ['SAMME.R'],
              "n_estimators" :[2],
              "learning_rate":  [0.1]}

gs_ada = GridSearchCV(ada,param_grid = ada_param_best, cv=k_fold, scoring="accuracy", n_jobs= 4, verbose = 1)

gs_ada.fit(X_train, y_train)

# best_param = gs_ada.best_params_

Y_pred_ada = gs_ada.predict(X_test)

acc_log = round(gs_ada.score(X_valid, y_valid) * 100, 2)

ada_best = gs_ada.best_estimator_

print(acc_log)
# print(best_param)


# Random Forest

In [ ]:
rfc = RandomForestClassifier()

best_param_grid = {'n_estimators': [2000], 'min_samples_split': [10], 'min_samples_leaf': [4], 'max_features': ['auto'], 'max_depth': [50], 'bootstrap': [False]}

gs_rfc = GridSearchCV(rfc,param_grid = best_param_grid, cv=k_fold, scoring="accuracy", n_jobs= 4, verbose = 1)
gs_rfc.fit(X_train, y_train)

Y_pred_rfc = gs_rfc.predict(X_test)
acc_log = round(gs_rfc.score(X_valid, y_valid) * 100, 2)

rfc_best = gs_rfc.best_estimator_


print(acc_log)
# print(gs_rfc.best_params_)


# Gaussian Naive Bayes

In [ ]:
nbs = GaussianNB()
nbs.fit(X_train, y_train)
Y_pred_nbs = nbs.predict(X_test)
acc_log = round(nbs.score(X_valid, y_valid) * 100, 2)
acc_log

# Support Vector Machines

In [ ]:
svc = SVC(probability=True)

# svc_param_grid = {'kernel': ['rbf'], 
#                   'gamma': [ 0.01, 0.1, 1,10],
#                  'C': [1, 10, 50, 100,200,300, 1000]}

best_param = {'kernel': ['rbf'], 
                  'gamma': [ 0.01],
                 'C': [50]}

gs_svc = GridSearchCV(svc,param_grid = best_param, cv=k_fold, scoring="accuracy", n_jobs= 4, verbose = 1)
gs_svc.fit(X_train, y_train)

Y_pred_svc = gs_svc.predict(X_test)
acc_log = round(gs_svc.score(X_valid, y_valid) * 100, 2)

svc_best = gs_svc.best_estimator_


print(acc_log)
# print(gs_svc.best_params_)

# Logistic Regression

In [ ]:
lgr = LogisticRegression(max_iter=200)
lgr.fit(X_train, y_train)
Y_pred_lgr = lgr.predict(X_test)
acc_log = round(lgr.score(X_valid, y_valid) * 100, 2)
acc_log

# XGBoost

In [ ]:
xgb = XGBClassifier(n_estimators=1000, learning_rate=0.1)

# params = {
#         'min_child_weight': [1, 5, 10],
#         'gamma': [0.5, 1, 1.5, 2, 5],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.6, 0.8, 1.0],
#         'max_depth': [3, 4, 5]
#         }
params_best = {
        'min_child_weight': [5],
        'gamma': [1],
        'subsample': [1.0],
        'colsample_bytree': [0.8],
        'max_depth': [5]
        }

gs_xgb = GridSearchCV(xgb,param_grid=params_best, cv=k_fold, scoring="accuracy", n_jobs= 4, verbose = 1)

gs_xgb.fit(X_train, y_train)

Y_pred_xgb = gs_xgb.predict(X_test)
acc_log = round(gs_xgb.score(X_valid, y_valid) * 100, 2)

xgb_best = gs_xgb.best_estimator_


print(acc_log)
# print(gs_xgb.best_params_)


# xgb.fit(X_train, y_train)
# Y_pred_xgb = xgb.predict(X_test)
# acc_log = round(xgb.score(X_valid, y_valid) * 100, 2)
# acc_log

# Ensemble modeling

In [ ]:
votingC = VotingClassifier(estimators=[('rfc', rfc_best),
('svc', svc_best), ('adac',ada_best),('xgb',xgb_best)], voting='soft', n_jobs=4)

votingC.fit(X, y)

In [ ]:
voting_c_pred = votingC.predict(X_test)

# Submission

In [ ]:
output = pd.DataFrame({'PassengerId': test_df["PassengerId"], 'Survived': voting_c_pred})
output.to_csv('my_submission_3.csv', index=False)

In [ ]:
result = pd.read_csv('my_submission_3.csv')
result.head()